# PipelineEndpoint Demo

In [1]:
from azureml.pipeline.wrapper import Module, dsl
from azureml.pipeline.wrapper._pipeline_endpoint import PipelineEndpoint
from workspace_helpers import setup_default_workspace, load_or_register_dataset

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 0.1.0.0 (c:\users\liyw\pycharmprojects\azuremlcli\src\azureml-core), Requirement.parse('azureml-core~=1.9.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 0.1.0.0 (c:\users\liyw\pycharmprojects\azuremlcli\src\azureml-core), Requirement.parse('azureml-core~=1.9.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.run:StepRun._from_dto with exception (azureml-core 0.1.0.0 (c:\users\liyw\pycharmprojects\azuremlcli\src\azureml-core), Requirement.parse('azureml-core~=1.9.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto 

In [12]:
workspace = setup_default_workspace()

In [9]:
experiment_name = "TestPipelineEndpoint"
module_func = Module.load(workspace, namespace='azureml', name='Select Columns in Dataset')
test_pipeline_endpoint_name = "test_pipeline_endpoint"
test_pipeline_endpoint_id = "b27eb28c-da34-4064-9c13-e515fdb2c71b"

blob_input_data = load_or_register_dataset(workspace, name='Automobile_price_data_(Raw)',
                                           datastore_name='azureml_globaldatasets',
                                           path='GenericCSV/Automobile_price_data_(Raw)')
select_columns = "{\"isFilter\":true,\"rules\":[{\"exclude\":false,\"ruleType\":\"AllColumns\"}," \
                 "{\"exclude\":true,\"ruleType\":\"ColumnNames\",\"columns\":[\"normalized-losses\"]}]}"

@dsl.pipeline(name='test-pipeline', default_compute_target='aml-compute')
def test_pipeline(_select_columns):
    module1 = module_func(
        dataset=blob_input_data,
        select_columns=_select_columns
    )
    return module1.outputs

In [13]:
pipeline_parameter = {'_select_columns': select_columns}
pipeline = test_pipeline(**pipeline_parameter)
run = pipeline.submit(experiment_name=experiment_name)
published_pipeline = run.publish_to_endpoint(pipeline_endpoint_name=test_pipeline_endpoint_name,
                                           name="test_pipeline_endpoint",
                                           use_existing_pipeline_endpoint=True,
                                           set_as_default=True)

Submitted PipelineRun e29f78ab-21d3-41d6-a2da-c7cf207b66e7


method publish_to_endpoint: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/TestPipelineEndpoint/runs/e29f78ab-21d3-41d6-a2da-c7cf207b66e7?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS
Using existing pipeline endpoint "test_pipeline_endpoint"


### 1. Get pipeline endpoint
Get the PipelineEndpoint by name or ID

#### get by id

In [16]:
endpoint_got_by_id = PipelineEndpoint.get(workspace=workspace, endpoint_id=test_pipeline_endpoint_id)
print(type(endpoint_got_by_id))
print(endpoint_got_by_id.name)

<class 'azureml.pipeline.wrapper._pipeline_endpoint.PipelineEndpoint'>
test_pipeline_endpoint


#### get by name

In [17]:
endpoint_got_by_name = PipelineEndpoint.get(workspace=workspace, endpoint_name=test_pipeline_endpoint_name)
print(type(endpoint_got_by_name))
print(endpoint_got_by_name.name)

<class 'azureml.pipeline.wrapper._pipeline_endpoint.PipelineEndpoint'>
test_pipeline_endpoint


#### get not-exist endpoint

In [19]:
PipelineEndpoint.get(workspace=workspace, endpoint_name="pipeline_endpoint_not_exist")

Exception: Pipeline Endpoint "pipeline_endpoint_not_exist" not exist.

### 2. Pipeline endpoint Submit
Submit default version of endpoint.

In [20]:
test_submit_experiment_name = "test_submit_by_pipeline_endpoint"
pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, endpoint_id=test_pipeline_endpoint_id)

run_submitted_by_pipeline_endpoint = pipeline_endpoint.submit(experiment_name=test_submit_experiment_name)

Submit pipeline test_pipeline_endpoint use default parameters {'_select_columns': '{"isFilter":true,"rules":[{"exclude":false,"ruleType":"AllColumns"},{"exclude":true,"ruleType":"ColumnNames","columns":["normalized-losses"]}]}'}
Submitted PipelineEndpointRun ed00c98f-19a8-4b6c-bc13-459f22f72b55
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/test_submit_by_pipeline_endpoint/runs/ed00c98f-19a8-4b6c-bc13-459f22f72b55?wsid=/subscriptions/4faaaf21-663f-4391-96fd-47197c630979/resourcegroups/DesignerTestRG/workspaces/DesignerTest-EUS


In [21]:
run_submitted_by_pipeline_endpoint.tags["azureml.pipelineid"]

'baee7db6-b947-4b13-950e-c032f9285e44'

In [22]:
pipeline_endpoint._default_pipeline_id

'baee7db6-b947-4b13-950e-c032f9285e44'

### 3. List
List active PipelineEndpoints in the current workspace.

In [23]:
pipeline_endpoints_list = PipelineEndpoint.list(workspace=workspace, active_only=False)

In [24]:
pipeline_endpoints_list

In [48]:
print("pipeline endpoints name list:") 
[endpoint.name for endpoint in pipeline_endpoints_list]

pipeline endpoints name list:


['test_published_pipeline',
 'test_publish_to_pipeline',
 'test_pipeline_endpoint',
 'test_endpoint_publish_endpoint',
 'pipeline_export_test',
 'test',
 'test-sub-graph',
 'Pipeline-Created-on-7-6-2020111',
 'Pipeline-Created-on-7-6-2020',
 'Sample 3: test2',
 'Sample 3: Binary Classification with Feature Selection - Income Prediction',
 'Sample 1: Regression - Automobile Price Prediction (Basic)']

### 4. Enable & Disable

In [29]:
pipeline_endpoint = PipelineEndpoint.get(workspace=workspace, endpoint_id=test_pipeline_endpoint_id)
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


In [30]:
pipeline_endpoint.disable()
print(pipeline_endpoint.status)

Disabled


In [31]:
pipeline_endpoint.enable()
print(pipeline_endpoint.status)

Active


### 5. Get Published Pipeline by version
Get the pipeline of a specified version or the default; throws an exception if the version is not found.

In [32]:
pipeline_version = '0'
pipeline_got = pipeline_endpoint.get_pipeline(version=pipeline_version)

In [33]:
pipeline_got

In [37]:
pipeline_default = pipeline_endpoint.get_pipeline()
print('pipeline_default.id:', pipeline_default.id)
print('pipeline_endpoint default version:', pipeline_endpoint._default_pipeline_id)

pipeline_default.id baee7db6-b947-4b13-950e-c032f9285e44
pipeline_endpoint default version: baee7db6-b947-4b13-950e-c032f9285e44


In [38]:
pipeline_version_not_exist = 'not_exist'
pipeline_endpoint.get_pipeline(version=pipeline_version_not_exist)

ValueError: Pipeline version "not_exist" not found in PipelineEndpoint "b27eb28c-da34-4064-9c13-e515fdb2c71b"

### 6. Get all pipelines
Get list of PublishedPipelines in PipelineEndpoint.
active_only: Whether to return only active pipelines.

In [39]:
pipelines_list = pipeline_endpoint.get_all_pipelines(active_only=True)

In [44]:
pipelines_list[0]

### 7. pipeline version list property
class PipelineVersion(object):

    def __init__(self, version, name, id, description, is_default, status):
        self._version = version
        self._name = name
        self._id = id
        self._description = description
        self._is_default = is_default
        self._status = status

In [41]:
versions_list = pipeline_endpoint.pipeline_version_list

In [43]:
versions_list[0]